In [42]:
# Step 1: Import Libraries and Load the Model
import numpy as np
import keras
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model

In [43]:
import warnings
warnings.filterwarnings('ignore')

In [44]:
# Load word index and reverse index
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

In [45]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [53]:
import h5py
import json

model_path = "best_model.h5"

with h5py.File(model_path, "r+") as f:
    raw_config = f.attrs["model_config"]
    if isinstance(raw_config, bytes):  # decode if needed
        raw_config = raw_config.decode("utf-8")
    model_config = json.loads(raw_config)

    for layer in model_config["config"]["layers"]:
        if layer["class_name"] == "InputLayer":
            layer["config"].pop("batch_shape", None)

    f.attrs["model_config"] = json.dumps(model_config)

print("Patched model file saved.")

from keras.models import load_model

model = load_model("best_model.h5", compile=False, safe_mode=False)


Patched model file saved.


TypeError: Error when deserializing class 'Embedding' using config={'name': 'embedding', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'input_dim': 10000, 'output_dim': 128, 'embeddings_initializer': {'module': 'keras.initializers', 'class_name': 'RandomUniform', 'config': {'seed': None, 'minval': -0.05, 'maxval': 0.05}, 'registered_name': None}, 'embeddings_regularizer': None, 'activity_regularizer': None, 'embeddings_constraint': None, 'mask_zero': False}.

Exception encountered: Unknown dtype policy: 'DTypePolicy'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [ ]:
# Function to decode encoded reviews (optional)
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

In [ ]:
# Function to preprocess user input
def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]  # 2 = unknown
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return padded_review


In [ ]:
# Step 3: Prediction function
def predict_sentiment(review):
    preprocessed_input = preprocess_text(review)
    prediction = model.predict(preprocessed_input)
    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    return sentiment, prediction[0][0]

In [ ]:
example_review = "This movie was fantastic! The acting was great and the plot was thrilling."

sentiment,score=predict_sentiment(example_review)

print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')